In [1]:
import os 
import sys
import numpy as np
import argparse
from scipy.spatial.transform import Rotation as R 
import open3d as o3d 
# add catkin_ws context 
sys.path.append("/home/junting/catkin_ws/devel/lib/python3.9/site-packages")
sys.path.append("/home/junting/franka_ws/devel/lib/python3.9/site-packages")

from src.lmp import *
from src.env.true_grounding_env import TrueGroundingEnv
from src.config import cfg_tabletop
import rospy 
import rospkg
import jupyros as jr

from std_msgs.msg import String, Header
from geometry_msgs.msg import PoseStamped, Pose, Point, Quaternion


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Failed to import pyassimp, see https://github.com/ros-planning/moveit/issues/86 for more info


In [2]:
########################################
# initialize environment
########################################
rospy.init_node('eval_code', log_level=rospy.DEBUG)
# get package root path 
pkg_root = rospkg.RosPack().get_path('instruct_to_policy')

# setup environment
env = TrueGroundingEnv(cfg_tabletop)
# env.reset()


[DEBUG] [1703077824.241825, 0.000000]: init_node, name[/eval_code], pid[1624613]
[DEBUG] [1703077824.243805, 0.000000]: binding to 0.0.0.0 0
[DEBUG] [1703077824.244740, 0.000000]: bound to 0.0.0.0 43399
[DEBUG] [1703077824.245363, 0.000000]: connecting to junting-PC 33927
[DEBUG] [1703077824.246316, 0.000000]: ... service URL is rosrpc://junting-PC:43399
[DEBUG] [1703077824.247374, 0.000000]: [/eval_code/get_loggers]: new Service instance
[DEBUG] [1703077824.248933, 0.000000]: ... service URL is rosrpc://junting-PC:43399
[DEBUG] [1703077824.249866, 0.000000]: [/eval_code/set_logger_level]: new Service instance
[DEBUG] [1703077824.388994, 909.826000]: connecting to junting-PC 33927
[DEBUG] [1703077824.393616, 909.830000]: connecting to junting-PC 52705
[INFO] [1703077824.397855, 909.834000]: camera_left: Waiting for camera_left/color/camera_info...
[DEBUG] [1703077824.405423, 909.841000]: connecting to junting-PC 52705
[DEBUG] [1703077824.406861, 909.842000]: connecting to junting-PC 33

## Ground truth perception and 3D fusion 

In [3]:
# 3D fusion 
from src.perception.scene_manager import SceneManager
sensor_data = env.get_sensor_data()
sensor_data['detections_list'] = [{},{},{}]

scene_manager = SceneManager()
scene_manager.update_fusion(sensor_data)

In [4]:
# get cropped point cloud
# drawer_bbox = env.get_3d_bbox('cabinet.drawer0')
# drawer_pcd = scene_manager.scene_tsdf_full.crop_cloud(
#     crop_center=(drawer_bbox[:3] + drawer_bbox[3:]) / 2,
#     crop_size=(drawer_bbox[3:] - drawer_bbox[:3])
# )
# cabinet_bbox = env.get_3d_bbox('cabinet')
# cabinet_pcd = scene_manager.scene_tsdf_full.crop_cloud(
#     crop_center=(cabinet_bbox[:3] + cabinet_bbox[3:]) / 2,
#     crop_size=(cabinet_bbox[3:] - cabinet_bbox[:3])
# )

In [5]:
import open3d as o3d 
# o3d.io.write_point_cloud(os.path.join(pkg_root, 'data', 'drawer.pcd'), drawer_pcd)
# o3d.io.write_point_cloud(os.path.join(pkg_root, 'data', 'cabinet.pcd'), cabinet_pcd)

## Pick and Place

In [ ]:
env.reset()

In [ ]:
pose_msg = env.parse_adaptive_shape_grasp_pose("apple")

In [ ]:
rot_mat = R.from_quat([pose_msg.orientation.x, pose_msg.orientation.y, pose_msg.orientation.z, pose_msg.orientation.w]).as_matrix()
translation = np.array([pose_msg.position.x, pose_msg.position.y, pose_msg.position.z])

depth = 0.05

pregrasp_offset_local = np.array([0, 0, -0.15]).astype(np.float32)
# predicted gripper center is 0.02m above the gripper tip
approach_offset_local = np.array([0, 0, depth - 0.02 ]).astype(np.float32)
pregrasp_position = translation + rot_mat @ pregrasp_offset_local
approach_position = translation + rot_mat @ approach_offset_local

pregrasp_pose = Pose(position=Point(*pregrasp_position), orientation=pose_msg.orientation)
approach_pose = Pose(position=Point(*approach_position), orientation=pose_msg.orientation)

In [ ]:
env.open_gripper()

In [ ]:
env.publish_goal_to_marker(pregrasp_pose)
env.move_to_pose(pregrasp_pose)

In [ ]:
env.publish_goal_to_marker(approach_pose)
env.move_to_pose(approach_pose)

In [ ]:
env.close_gripper(width=0.05, force=30)

In [ ]:
env.attach_object("apple")

In [ ]:
place_pose = env.parse_place_pose(object_name="apple", receptacle_name="white_ceramic_plate")

In [ ]:
env.move_to_pose(place_pose)
env.open_gripper()

In [ ]:
env.detach_object("apple")

## Open Drawer

In [ ]:
jr.publish('/rviz/moveit/move_marker/goal_panda_hand_tcp', PoseStamped)

In [ ]:
env.reset()
# env.open_gripper()

In [ ]:
# env.get_obj_name_list()
# [bbox.object_id for bbox in env.gazebo_gt_bboxes]
env.get_3d_bbox("cabinet.handle_0")

In [ ]:
grasp_pose = env.parse_adaptive_shape_grasp_pose(object_name="cabinet.handle_0")
print(grasp_pose)

In [ ]:
env.publish_goal_to_marker(grasp_pose)

In [ ]:
env.grasp(grasp_pose)
# env.move_to_pose(grasp_pose)

In [ ]:
env.close_gripper()

In [ ]:
# generate a horizontal trajectory to open the drawer
grasp_position = np.array([grasp_pose.position.x, grasp_pose.position.y, grasp_pose.position.z])
pull_position = grasp_position + np.array([0.2, 0, 0]).astype(float)
pull_pose = Pose(position=Point(*pull_position), orientation=grasp_pose.orientation)

In [ ]:
env.move_to_pose(pull_pose)

In [ ]:
env.open_gripper()

In [ ]:
# test different grasp depth 
pose_msg = env.parse_adaptive_shape_grasp_pose(object_name="apple")

rot_mat = R.from_quat([pose_msg.orientation.x, pose_msg.orientation.y, pose_msg.orientation.z, pose_msg.orientation.w]).as_matrix()
translation = np.array([pose_msg.position.x, pose_msg.position.y, pose_msg.position.z])

depth = 0.05

pregrasp_offset_local = np.array([0, 0, -0.15]).astype(np.float32)
# predicted gripper center is 0.02m above the gripper tip
approach_offset_local = np.array([0, 0, depth - 0.02 ]).astype(np.float32)
pregrasp_position = translation + rot_mat @ pregrasp_offset_local
approach_position = translation + rot_mat @ approach_offset_local

pregrasp_pose = Pose(position=Point(*pregrasp_position), orientation=pose_msg.orientation)
approach_pose = Pose(position=Point(*approach_position), orientation=pose_msg.orientation)
 
env.open_gripper()
 
# env.publish_goal_to_marker(pregrasp_pose)
env.move_to_pose(pregrasp_pose)
# env.publish_goal_to_marker(approach_pose)
env.move_to_pose(approach_pose)
env.close_gripper(width=0.05, force=30)
env.attach_object("apple")


In [ ]:
env.reset()

## Grasp preference

In [19]:
from graspnetAPI import GraspGroup, Grasp
from grasp_detection.msg import Grasp as GraspMsg
from src.grasp_detection.utils import select_grasp_by_preference

# reload src.grasp_detection.utils 
import importlib
importlib.reload(sys.modules['src.grasp_detection.utils'])
from src.grasp_detection.utils import select_grasp_by_preference

In [6]:
object_name = "white_and_pink_box"


In [7]:
# ground truth preference of object 

# preferred_position is the center of the object
preferred_position = env.get_object_center_position(object_name)

# preferred_orientation is table surface 
preferred_plane_normal = np.array([0, 0, 1])


[DEBUG] [1702551930.771895, 137.547000]: connecting to junting-PC 34991
[DEBUG] [1702551930.774374, 137.549000]: connecting to junting-PC 34991


In [8]:
# Manually call grasp detection model to get list of grasps

object_bbox = env.get_3d_bbox(object_name)    
sensor_data = env.get_sensor_data()

object_pcd = scene_manager.scene_tsdf_full.crop_cloud(
    crop_center=(object_bbox[:3] + object_bbox[3:]) / 2,
    crop_size=(object_bbox[3:] - object_bbox[:3])
)

bbox_center = (object_bbox[:3] + object_bbox[3:]) / 2
bbox_size = object_bbox[3:] - object_bbox[:3]
        
data = {
    # 'detections_list': detections_list,
    'bboxes_3d_dict':{
        object_name:{'center': bbox_center, 'size': bbox_size}
    }
}
data.update(sensor_data)

# call grasp detection service
grasp_candidates: List[GraspMsg] = env.grasp_model.predict(data)

[INFO] [1702551932.435929, 139.185000]: Sending perception data to grasp detection service
[DEBUG] [1702551932.441211, 139.204000]: connecting to junting-PC 35057


In [20]:
# Manually select a grasp from the list of grasps
grasp_idx, weighted_score = select_grasp_by_preference(grasp_candidates, preferred_position=preferred_position, preferred_plane_normal=preferred_plane_normal)

In [21]:
# visualize all grasp candidates with object point cloud 
grasp_o3d_meshes = []

# rotate grasp orientation from gazebo gripper frame to anygrasp gripper frame
rot_anygrasp2gazebo = np.array([[0,0,1],[0,1,0],[-1,0,0]])
rot_gazebo2anygrasp = np.linalg.inv(rot_anygrasp2gazebo)

for i, grasp_msg in enumerate(grasp_candidates):
    quat = np.array([grasp_msg.grasp_pose.orientation.x, grasp_msg.grasp_pose.orientation.y, grasp_msg.grasp_pose.orientation.z, grasp_msg.grasp_pose.orientation.w])
    rotation_matrix = R.from_quat(quat).as_matrix()
    rotation_matrix = rotation_matrix @ rot_gazebo2anygrasp
    translation = np.array([grasp_msg.grasp_pose.position.x, grasp_msg.grasp_pose.position.y, grasp_msg.grasp_pose.position.z])
    # [score, width, height, depth, rotation_matrix, translation, object_id]
    grasp = Grasp(
        *[grasp_msg.grasp_score, grasp_msg.grasp_width, 0.02, grasp_msg.grasp_depth, rotation_matrix, translation, i]
    )
    grasp_o3d_meshes.append(grasp.to_open3d_geometry())
    
# change the selected grasp color to red 
grasp_o3d_meshes[grasp_idx].paint_uniform_color([1,0,0])

o3d.visualization.draw_geometries([object_pcd, *grasp_o3d_meshes])    

In [23]:
# predict the grasp pose of white_and_pink_box with anygrasp
pose_msg = env.parse_adaptive_shape_grasp_pose(object_name=object_name, preferred_position=preferred_position, preferred_plane_normal=preferred_plane_normal)
env.publish_goal_to_marker(pose_msg)

[INFO] [1702556677.544106, 4876.112000]: Sending perception data to grasp detection service
[DEBUG] [1702556677.550433, 4876.132000]: connecting to junting-PC 35057
[DEBUG] [1702556679.132464, 4877.621000]: Waiting for rviz to update


## Joint Prediction 

In [3]:
env.get_gt_bbox('cabinet.handle_0')

[DEBUG] [1703077835.437317, 919.205000]: connecting to junting-PC 33927


([-1.1429746065220936, -0.005033182851874846, 1.4529604760174615],
 [0.06472280556051602, 0.4073600424866717, 0.02192639470308233])

In [4]:
# get handle 0 position 
handle_0_position = env.get_object_center_position("cabinet.handle_0")
print(handle_0_position)

[DEBUG] [1703077835.714537, 919.425000]: connecting to junting-PC 33927
[-1.14297333 -0.00503446  1.45295916]


In [5]:
env.get_object_joint_axis(obj_name="cabinet", position=handle_0_position, type="any")

[DEBUG] [1703077836.787005, 920.244000]: connecting to junting-PC 33927
[DEBUG] [1703077836.800585, 920.253000]: connecting to junting-PC 33927


array([ 9.99999999e-01, -4.65130437e-05, -9.34212915e-07])